In [8]:
import requests
from bs4 import BeautifulSoup

# URL of a player listing page (can be changed to any relevant listing page)
url = "https://www.espncricinfo.com/india/content/player/caps.html?country=6;class=11"  # Example for India T20I players

# Send a request to the URL
response = requests.get(url)

# Initialize BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all player profile links on the page
# Player links generally have '/player/' in the URL
players = soup.find_all('a', href=True)
print(f"Total players found: {len(players)}")
# Extract player IDs and names
for player in players:
    link = player['href']
    if '/player/' in link:  # Filter player profile links
        player_name = player.text.strip()
        player_id = link.split('/')[-1].split('.')[0]  # Extracting player ID from the URL
        print(f"Player Name: {player_name}, Player ID: {player_id}")

Total players found: 0


In [11]:
import espncricinfo

In [15]:
from espncricinfo.series import Series
s = Series('18018')
s.name

ModuleNotFoundError: No module named 'grequests'

In [17]:
import requests

# URL of the API endpoint you want to access
url = "https://hs-consumer-api.espncricinfo.com/v1/pages/matches/current?lang=en&latest=true"

# Define headers to mimic a browser request
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'en-IN,en;q=0.9,hi-IN;q=0.8,hi;q=0.7,en-GB;q=0.6,en-US;q=0.5,te;q=0.4',
    'origin': 'https://www.espncricinfo.com',
    'referer': 'https://www.espncricinfo.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Mobile Safari/537.36',
    # Add the authorization token header if required
    # 'x-hsci-auth-token': 'your-auth-token'
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check for successful response
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    print(data)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


Failed to fetch data. Status code: 403
